In [1]:
# Prep checking
import pandas as pd

df_raw = pd.read_csv('data/diabetic_data.csv')
df_prep = pd.read_csv('data/diabetic_data_preprocessed.csv')

race_cols = [col for col in df_prep.columns if col.startswith('race')]
df_prep.loc[df_raw.race == '?', race_cols]
# tingues en compte que ara ja he "imputat" algun per casos del mateix usuari

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other
19,0,0,0,0,0
20,0,0,0,0,0
21,0,0,0,0,0
54,0,0,0,0,0
65,0,0,0,0,0
...,...,...,...,...,...
101659,0,0,0,0,0
101671,0,0,0,0,0
101702,0,0,0,0,0
101715,0,0,0,0,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier

norace = df_prep.drop(race_cols, axis=1)
train = norace.loc[df_raw.race != '?']
test = norace.loc[df_raw.race == '?']
y_train = df_raw.loc[df_raw.race != '?', 'race']
y_test = df_raw.loc[df_raw.race == '?', 'race'] 

model = RandomForestClassifier()
model.fit(train, y_train)  # Usar variables relevantes

# Predecir missings
missings = model.predict(test)

In [ ]:
# Imputar
# Convert the predicted missing values to one-hot encoding
imputed_one_hot = pd.get_dummies(missings, prefix='race', dtype=int)

# Ensure all race columns are present in the one-hot encoded dataframe
for col in race_cols:
  if col not in imputed_one_hot.columns:
    imputed_one_hot[col] = 0

# Reorder columns to match the original dataframe
imputed_one_hot = imputed_one_hot[race_cols]

# Assign the one-hot encoded values back to the original dataframe
df_prep.loc[df_raw.race == '?', race_cols] = imputed_one_hot.values

In [21]:
df_prep.loc[df_raw.race == '?', race_cols].sum()


race_AfricanAmerican     113
race_Asian                 0
race_Caucasian          2160
race_Hispanic              0
race_Other                 0
dtype: object